In [1]:
#para
import random
import time
n = 16
R = 5
Field.<a> = GF(2**16+1, modulus="primitive", proof=false)
nc = 1024
secret_key = [i for i in range(n)]
print(secret_key)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [2]:
def addition_for_vec(vec1, vec2):
    return [vec1[i]+vec2[i] for i in range(n)]

def matrix_mul_vec(mat, vec):
    result_vec = []
    for i in range(n):
        tmp_result = 0
        for j in range(n):
            tmp_result += mat[i][j]*vec[j]
        result_vec.append(tmp_result)
    return result_vec

def generate_affine_layer(r):
    set_random_seed(nc+r)
    vec_b = []
    for i in range(n):
        b_element = Field.random_element()
        if b_element != 0:
            vec_b.append(b_element)
    
    vec_a = []
    L_matrix = []
    for i in range(n):
        a_element = Field.random_element()
        if a_element != 0:
            vec_a.append(a_element)
    
    for i in range(n):
        new_row = []
        for j in range(n):
            if j <= i:
                new_row.append(vec_a[i-j])
            else:
                new_row.append(a*vec_a[n+i-j])
        L_matrix.append(new_row)
    return L_matrix, vec_b

# def generate_final_constant(r):
#     set_random_seed(nc+r)
#     vec_b = []
#     for i in range(n):
#         b_element = Field.random_element()
#         if b_element != 0:
#             vec_b.append(b_element)
#     return vec_b

def kai_operation(input_state):
    output_state = []
    for i in range(n):
        new_elem = input_state[i] + input_state[(i+2)%n] + input_state[(i+1)%n]*input_state[(i+2)%n]
        output_state.append(new_elem)
    return output_state

def Masta_encryption(k, R):
    round_state = k
    
    for r in range(R):
        M, b = generate_affine_layer(r)
        round_state = kai_operation(addition_for_vec(matrix_mul_vec(M, round_state), b))
    
    M, b = generate_affine_layer(R)
    round_state = addition_for_vec(matrix_mul_vec(M, round_state), b)
    round_state = addition_for_vec(round_state, secret_key)
    return round_state


def Fault_Masta_encryption(pos, delta, k, R):
    round_state = k
    
    for r in range(R-1):
        M, b = generate_affine_layer(r)
        round_state = kai_operation(addition_for_vec(matrix_mul_vec(M, round_state), b))
    
#     x = round_state
#     print("S:", round_state)
    M1,b1 = generate_affine_layer(R-1)
    M2,b2 = generate_affine_layer(R)
    fault_round_state = []
    for i in range(n):
        if i == pos:
            fault_round_state.append(delta+round_state[i])
        else:
            fault_round_state.append(round_state[i])
#     print("fault S:", fault_round_state)
#     check_delta = [fault_round_state[i] - round_state[i] for i in range(n)]
#     print("D(S):", check_delta)
    
#     round_state = matrix_mul_vec(M1, round_state)
#     fault_round_state = matrix_mul_vec(M1, fault_round_state)
#     check_delta = [fault_round_state[i] - round_state[i] for i in range(n)]
#     print("D(M1*S):", check_delta)
    
#     round_state = addition_for_vec(round_state, b1)
#     fault_round_state = addition_for_vec(fault_round_state, b1)
#     check_delta = [fault_round_state[i] - round_state[i] for i in range(n)]
#     print("D(M1*S+b1):", check_delta) 
#     print("X:", round_state)
    
#     round_state = kai_operation(round_state)
#     fault_round_state = kai_operation(fault_round_state)
#     check_delta = [fault_round_state[i] - round_state[i] for i in range(n)]
#     print("D(y):", check_delta)
    
#     round_state = matrix_mul_vec(M2, round_state)
#     fault_round_state = matrix_mul_vec(M2, fault_round_state)
#     print(matrix_mul_vec(M2, check_delta))
#     check_delta = [fault_round_state[i] - round_state[i] for i in range(n)]
#     print("D(M2*y):", check_delta)
    
#     print(matrix_mul_vec(Matrix(Field, M2).inverse_of_unit(), check_delta))
    
#     round_state = addition_for_vec(round_state, b2)
#     fault_round_state = addition_for_vec(fault_round_state, b2)
#     check_delta = [fault_round_state[i] - round_state[i] for i in range(n)]
#     print("D(M2*y+b2):", check_delta) 
    
#     round_state = addition_for_vec(round_state, k)
#     fault_round_state = addition_for_vec(fault_round_state, k)
#     check_delta = [fault_round_state[i] - round_state[i] for i in range(n)]
#     print("D(z):", check_delta)     
#     X = addition_for_vec(matrix_mul_vec(M1, round_state), b1)
#     print("X:", X)
    
    fault_round_state = kai_operation(addition_for_vec(matrix_mul_vec(M1, fault_round_state), b1))
    
    fault_round_state = addition_for_vec(matrix_mul_vec(M2, fault_round_state), b2)    
    fault_round_state = addition_for_vec(fault_round_state, k)
    return fault_round_state


def Construct_and_solve(Ds, Dz, R):
    M1,b1 = generate_affine_layer(R-1)
    M2,b2 = generate_affine_layer(R)
    
    Dx = matrix_mul_vec(M1, Ds)
    Dy = matrix_mul_vec(Matrix(Field, M2).inverse_of_unit(), Dz)
    
#     print(Ds, Dz)
    print("Dx", Dx)
    print("Dy", Dy)
    
    coefficient_matrix = [[0 for j in range(n)] for i in range(n)]
    bias_vec = [0 for i in range(n)]
    
    for i in range(n):
        coefficient_matrix[i][(i+1)%n] = Dx[(i+2)%n]
        coefficient_matrix[i][(i+2)%n] = Dx[(i+1)%n]
        bias_vec[i] = Dy[i] - Dx[i] - Dx[(i+2)%n] - Dx[(i+1)%n]*Dx[(i+2)%n]
    
    A = Matrix(Field, coefficient_matrix)
    b = vector(Field, bias_vec)
    
    print(A)
    print(A.rank())
    print(b)
#     print(matrix_mul_vec(A, x))
    
    x = A.solve_right(b)
    print(x)
    return x

def Construct_and_solve_v2(Ds, Dz, R):
    M1,b1 = generate_affine_layer(R-1)
    M2,b2 = generate_affine_layer(R)
    
    Dx = matrix_mul_vec(M1, Ds)
    coefficient_matrix = [[0 for j in range(n+1)] for i in range(n)]
    for i in range(n):
        coefficient_matrix[i][(i+1)%n] = Dx[(i+2)%n]
        coefficient_matrix[i][(i+2)%n] = Dx[(i+1)%n]
        coefficient_matrix[i][n] = Dx[i] + Dx[(i+2)%n] + Dx[(i+1)%n]*Dx[(i+2)%n]
        
    new_coef = Matrix(Field, M2)*Matrix(Field, coefficient_matrix)
    for i in range(n):
        new_coef[i, n] = Dz[i]-new_coef[i][n]
        
    A_matrix = [new_coef[i][:n] for i in range(n)]
    b_vec = [new_coef[i][n] for i in range(n)]
    r1 = Matrix(Field, A_matrix).rank() 
    r2 = new_coef.rank()
    if r1 < r2:
        return 0
    elif r1 == r2:
        if r1 == n:
            return 1
        else:
            return 2
        
def Construct_and_solve_v3(Ds, Dz, R, z, test_key):
    # In practice, we check key with Masta_encryption(). Here, we use test_key for simplicity.
    M1,b1 = generate_affine_layer(R-1)
    M2,b2 = generate_affine_layer(R)
    
    Dx = matrix_mul_vec(M1, Ds)
    coefficient_matrix = [[0 for j in range(n+1)] for i in range(n)]
    for i in range(n):
        coefficient_matrix[i][(i+1)%n] = Dx[(i+2)%n]
        coefficient_matrix[i][(i+2)%n] = Dx[(i+1)%n]
        coefficient_matrix[i][n] = Dx[i] + Dx[(i+2)%n] + Dx[(i+1)%n]*Dx[(i+2)%n]
        
    new_coef = Matrix(Field, M2)*Matrix(Field, coefficient_matrix)
    for i in range(n):
        new_coef[i, n] = Dz[i]-new_coef[i][n]
        
    A_matrix = [new_coef[i][:n] for i in range(n)]
    b_vec = [new_coef[i][n] for i in range(n)]
    r1 = Matrix(Field, A_matrix).rank() 
    r2 = new_coef.rank()
    if r1 < r2:
        return 0
    elif r1 == r2:
        print(Ds)
        if r1 == n:
            result_x = Matrix(Field, A_matrix).solve_right(vector(Field, b_vec))
            tmp = addition_for_vec(matrix_mul_vec(M2, result_x), b2)
            for i in range(n):
                if z[i] - tmp[i] - test_key[i] != 0:
                    return 0
            print("Find the right key!")
            return 1
        else:
            t1 = time.time()
            new_coef2 = new_coef.echelon_form()
            x_coef = [new_coef2[i][n-1] for i in range(n)]
            b_coef = [new_coef2[i][n] for i in range(n)]
#             print(new_coef2)
#             print(x_coef)
#             print(b_coef)
            for x16 in range(2**16+1):
                result_x = [b_coef[i] - x16*x_coef[i] for i in range(n-1)]
                result_x.append(x16)
                tmp = addition_for_vec(matrix_mul_vec(M2, kai_operation(result_x)), b2)
#                 if x16 == 29042:
#                     print(result_x)
                count = 0
                while count < n:
                    if z[count] - tmp[count] - test_key[count] != 0:
                        break
                    count += 1
#                 if count != 0:
#                     print(x16, result_x, count, [z[i]-tmp[i] for i in range(n)])
                if count == n:
                    print("Find the right key!")
                    print("Right key =", [z[i]-tmp[i] for i in range(n)])
                    print(result_x)
                    return 1
            return 0

In [7]:
random.seed(0)
delta_pos = random.randint(0, 15)
delta_val = random.randint(1, 2**16)
print(delta_pos, delta_val)
z = Masta_encryption(secret_key, R)
fault_z = Fault_Masta_encryption(delta_pos, delta_val, secret_key, R)
print("original output:",z)
print("faulty output:", fault_z)
delta_z = [fault_z[i] - z[i] for i in range(n)]
t1 = time.time()
for guess_pos in range(n):
    delta_s = [0 for i in range(n)]
    flag = 0
    for guess_val in range(1, 2**16+1):
        delta_s[guess_pos] = guess_val
        flag = Construct_and_solve_v3(delta_s, delta_z, R, z, secret_key)
        if flag:
            break
    print(guess_pos, time.time()-t1)
    if flag:
        break

12 55126
X: [20430, 16747, 31161, 63336, 17135, 53803, 29180, 2923, 23664, 43457, 57437, 45286, 44185, 52726, 55589, 49126]
original output: [11633, 62040, 42342, 41789, 30059, 52016, 23209, 617, 4624, 55970, 4482, 38094, 21988, 2799, 24648, 51935]
faulty output: [15563, 8511, 16157, 10003, 49596, 2349, 533, 64886, 33240, 65469, 38612, 3154, 12429, 23401, 47525, 24135]
[32932, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 368.731737613678
[0, 63132, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1 740.2858428955078
[0, 0, 18379, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2 1102.5437233448029
[0, 0, 0, 63680, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3 1463.6429517269135
[0, 0, 0, 0, 549, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4 1823.1954364776611
[0, 0, 0, 0, 0, 32463, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5 2192.6932995319366
[0, 0, 0, 0, 0, 0, 4228, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6 2557.63676404953
[0, 0, 0, 0, 0, 0, 0, 21182, 0, 0, 0, 0, 0, 0, 0, 0]
7 2916.177479982376
[0, 0, 0, 0, 0, 0, 0, 0, 58616, 0, 0, 0, 0, 0